In [115]:
import re

import numpy as np
import pandas as pd
import scipy.stats as stats
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import GridSearchCV, cross_val_score
from xgboost import XGBRegressor

In [116]:
df = pd.read_csv('data/data.csv')
df = df.set_index('AppID')

df

,Score,name,recent_percent,recent_count,all_percent,all_count,short_desc,long_desc,tags
AppID,,,,,,,,,
335300,6.0,DARK SOULS™ II: Scholar of the First Sin,83.0,1984.0,86.0,44838.0,DARK SOULS™ II: Scholar of the First Sin bring...,Gamers are in for a big surprise in DARK SOULS...,"['Souls-like', 'Dark Fantasy', 'RPG', 'Difficu..."
374320,6.0,DARK SOULS™ III,94.0,7519.0,94.0,180515.0,Dark Souls continues to push the boundaries wi...,Get the DARK SOULS™ III Season Pass now and ch...,"['Souls-like', 'Dark Fantasy', 'Difficult', 'R..."
570940,6.0,DARK SOULS™: REMASTERED,93.0,790.0,85.0,29345.0,"Then, there was fire. Re-experience the critic...","Then, there was fire. Re-experience the critic...","['Souls-like', 'Education', 'Dark Fantasy', 'A..."
435150,6.0,Divinity: Original Sin 2 - Definitive Edition,95.0,2310.0,95.0,109266.0,The critically acclaimed RPG that raised the b...,The Divine is dead. The Void approaches. And t...,"['Tactical RPG', 'Exploration', 'Story Rich', ..."
427520,6.0,Factorio,98.0,2187.0,98.0,97202.0,Factorio is a game about building and creating...,is a game in which you build and maintain fact...,"['Automation', 'Base Building', 'Resource Mana..."
...,...,...,...,...,...,...,...,...,...
1569090,NaN,Vivid Knight,90.0,114.0,92.0,884.0,Vivid Knight is a roguelike adventure game in ...,Explore a dungeon that changes with every play...,"['Roguelike Deckbuilder', 'Difficult', 'Choice..."
1582510,NaN,Mercenaries Blaze,71.0,32.0,75.0,166.0,“Mercenaries Blaze”is the 5th fantasy tactical...,A story about a corrupt kingdom and a fight fo...,"['RPG', 'Strategy', 'Turn-Based Tactics', 'Str..."
1599340,NaN,Lost Ark,97.0,1083.0,97.0,1083.0,Embark on an odyssey for the Lost Ark in a vas...,Embark on an odyssey for the Lost Ark in a vas...,"['Action', 'RPG', 'MMORPG', 'Action RPG', 'Adv..."


In [117]:
df = df[df['name'].notnull()]

percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})

missing_value_df

,column_name,percent_missing
Score,Score,90.815006
name,name,0.000000
recent_percent,recent_percent,0.000000
recent_count,recent_count,0.000000
all_percent,all_percent,0.000000
all_count,all_count,0.000000
short_desc,short_desc,10.478655
long_desc,long_desc,1.811125
tags,tags,0.000000


In [118]:
### Data Pre-process

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
model.max_seq_length = 10000

# Remove appids that were unable to pull information and fill null
df = df[df['name'].notnull()]

# Fill Null
df['short_desc'] = df['short_desc'].fillna('')
df['long_desc'] = df['short_desc'].fillna('')
df['tags'] = df['tags'].fillna('')

# Normalize ratings cols
df['recent_percent']=(df['recent_percent']-df['recent_percent'].mean())/df['recent_percent'].std()
df['recent_count']=(df['recent_count']-df['recent_count'].mean())/df['recent_count'].std()
df['all_percent']=(df['all_percent']-df['all_percent'].mean())/df['all_percent'].std()
df['all_count']=(df['all_count']-df['all_count'].mean())/df['all_count'].std()

# Process name
# Get name embeddings
df['name_emb'] = None
for idx, sentence in zip(df.index, df['name'].values):
    sentence_emb = model.encode(sentence)
    df.at[idx, 'name_emb'] = sentence_emb

# Explode name_emb to multiple cols
emb_len = df['name_emb'].values[0].shape[0]
emb_cols = [f'name_emb_{i}' for i in range(0, emb_len)]
df[emb_cols] = pd.DataFrame(df['name_emb'].tolist(), index=df.index)
df = df.drop(['name_emb'], axis=1)

# Process short_desc
# Get short_desc embeddings
df['short_desc_emb'] = None
for idx, sentence in zip(df.index, df['short_desc'].values):
    sentence_emb = model.encode(sentence)
    df.at[idx, 'short_desc_emb'] = sentence_emb

# Explode short_desc_emb to multiple cols
emb_len = df['short_desc_emb'].values[0].shape[0]
emb_cols = [f'short_desc_emb_{i}' for i in range(0, emb_len)]
df[emb_cols] = pd.DataFrame(df['short_desc_emb'].tolist(), index=df.index)
df = df.drop(['short_desc_emb'], axis=1)

# Process long_desc
# Get long_desc embeddings
df['long_desc_emb'] = None
for idx, sentence in zip(df.index, df['long_desc'].values):
    sentence_emb = model.encode(sentence)
    df.at[idx, 'long_desc_emb'] = sentence_emb

# Explode short_desc_emb to multiple cols
emb_len = df['long_desc_emb'].values[0].shape[0]
emb_cols = [f'long_desc_emb_{i}' for i in range(0, emb_len)]
df[emb_cols] = pd.DataFrame(df['long_desc_emb'].tolist(), index=df.index)
df = df.drop(['long_desc_emb'], axis=1)

# Process tags
# Get tags embeddings
df['tags_emb'] = None
for idx, sentence in zip(df.index, df['tags'].values):
    sentence_emb = model.encode(sentence)
    df.at[idx, 'tags_emb'] = sentence_emb

# Explode tags to multiple cols
emb_len = df['tags_emb'].values[0].shape[0]
emb_cols = [f'tags_emb_{i}' for i in range(0, emb_len)]
df[emb_cols] = pd.DataFrame(df['tags_emb'].tolist(), index=df.index)
df = df.drop(['tags_emb'], axis=1)

# Drop unneeded cols
df_proc = df.drop(['name', 'short_desc', 'long_desc', 'tags'], axis=1)

df_proc

/Users/mcminnra/miniconda3/envs/sgr/lib/python3.9/site-packages/pandas/core/frame.py:3636: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]


,Score,recent_percent,recent_count,all_percent,all_count,name_emb_0,name_emb_1,name_emb_2,name_emb_3,name_emb_4,...,tags_emb_374,tags_emb_375,tags_emb_376,tags_emb_377,tags_emb_378,tags_emb_379,tags_emb_380,tags_emb_381,tags_emb_382,tags_emb_383
AppID,,,,,,,,,,,,,,,,,,,,,
335300,6.0,0.110017,0.209128,0.220522,0.033753,-0.205884,0.530513,0.019095,0.098417,-0.168850,...,0.115262,0.345862,0.314601,0.397895,0.191249,0.155089,0.538477,-0.224874,-0.174770,0.102378
374320,6.0,0.641535,1.538971,0.617411,0.634029,-0.425299,0.070268,0.467150,-0.283747,-0.155813,...,0.126673,0.339611,0.323297,0.429290,0.262892,0.199058,0.464258,-0.147462,-0.180970,0.051384
570940,6.0,0.593215,-0.077743,0.170911,-0.034793,-0.411845,0.084661,0.643239,-0.140084,0.178792,...,0.235841,0.311501,0.388661,0.383162,0.072721,0.044172,0.577991,-0.172253,-0.140407,0.039733
435150,6.0,0.689854,0.287453,0.667022,0.318802,-0.083358,0.879001,0.190073,-0.143532,-0.288290,...,0.370163,0.379355,0.195057,0.410373,0.220993,0.058797,0.099029,0.168631,0.011805,0.039026
427520,6.0,0.834814,0.257901,0.815855,0.265427,-0.203326,-0.352310,0.348855,0.072503,0.431592,...,0.362606,0.404979,0.033295,-0.065764,0.200150,0.384024,-0.044277,-0.100570,0.045500,0.350224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1569090,NaN,0.448255,-0.240159,0.518188,-0.160713,0.290015,0.133307,0.122345,0.043767,0.076412,...,0.056362,0.609228,0.119564,0.231422,0.050099,0.187765,0.506191,-0.376130,0.170645,0.256374
1582510,NaN,-0.469821,-0.259861,-0.325199,-0.163889,-0.098121,0.447348,0.058343,-0.344018,0.370707,...,0.328313,0.634138,0.151306,0.273260,-0.119691,0.218492,-0.013120,-0.126463,0.119150,0.207751
1599340,NaN,0.786494,-0.007347,0.766244,-0.159832,0.179558,0.133478,0.584903,0.291660,0.171640,...,0.247450,0.369484,0.189386,0.351842,0.107656,0.282955,0.151401,-0.236875,0.094177,0.057548


In [120]:
### ML Model

# data split
df_model = df_proc[df_proc['Score'].notnull()]
X = df_model.drop(['Score'], axis=1)
y = df_model['Score']

df_pred = df_proc[df_proc['Score'].isnull()]
X_pred = df_pred.drop(['Score'], axis=1)

# Fit Model
model = XGBRegressor(
    max_depth=32,
    n_estimators=250,
    objective='reg:squarederror',
    random_state=42,
    verbosity=0,
    n_jobs=-1)
model.fit(X, y)

# Get Cross Val Score
scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=5)
print(f' Avg. MSE: {scores.mean():0.4f} (+/- {scores.std():0.4f})')

# pred
y_pred = model.predict(X_pred)

df_pred = pd.DataFrame({
    'AppID': X_pred.index.values,
    'Pred Score': y_pred
}).sort_values('Pred Score', ascending=False).set_index('AppID')
df_pred = df_pred.join(df[['name']], how='left')
df_pred = df_pred[['name', 'Pred Score']]

 Avg. MSE: -2.4472 (+/- 1.5821)


In [121]:
df_pred.head(25)

,name,Pred Score
AppID,,
3830,Psychonauts,5.446598
320,Half-Life 2: Deathmatch,5.433337
261550,Mount & Blade II: Bannerlord,5.409578
504230,Celeste,5.405684
560130,Pillars of Eternity II: Deadfire,5.395690
373420,Divinity: Original Sin - Enhanced Edition,5.383894
228280,Baldur's Gate: Enhanced Edition,5.338211
941570,Relicta,5.317822
420,Half-Life 2: Episode Two,5.288890


In [129]:
df_pred[df_pred['name'] == 'Dishonored 2']

,name,Pred Score
AppID,,
403640,Dishonored 2,4.187867


In [122]:
"""
Picked - Elderborn - Actual 2
373420	Divinity: Original Sin - Enhanced Edition	5.889911
230230	Divinity: Original Sin (Classic)	5.594720
258970	Gauntlet™ Slayer Edition	5.530334
727850	ELDERBORN	5.454527
236430	DARK SOULS™ II	5.429187
610180	The Jackbox Party Pack 4	5.410854
1086940	Baldur's Gate 3	5.398372
1090202	Destiny 2: Shadowkeep Digital Deluxe	5.387590
57300	Amnesia: The Dark Descent	5.353191
812140	Assassin's Creed® Odyssey	5.337534
288470	Fable Anniversary	5.336144
972660	Spiritfarer®	5.321304
466300	Planescape: Torment: Enhanced Edition	5.316258
434170	The Jackbox Party Pack 3	5.305679
973760	Thronebreaker: The Witcher Tales	5.304965
366090	Colony Survival	5.298052
995980	Fae Tactics	5.249742
1090200	Destiny 2: Shadowkeep	5.246860
55230	Saints Row: The Third	5.240653
320	Half-Life 2: Deathmatch	5.222818
471810	Death Squared	5.222415
374040	Portal Knights	5.198342
740130	Tales of Arise	5.192035
247240	Volgarr the Viking	5.189807
792710	Levelhead	5.186869
"""

"\nPicked - Elderborn - Actual 2\n373420\tDivinity: Original Sin - Enhanced Edition\t5.889911\n230230\tDivinity: Original Sin (Classic)\t5.594720\n258970\tGauntlet™ Slayer Edition\t5.530334\n727850\tELDERBORN\t5.454527\n236430\tDARK SOULS™ II\t5.429187\n610180\tThe Jackbox Party Pack 4\t5.410854\n1086940\tBaldur's Gate 3\t5.398372\n1090202\tDestiny 2: Shadowkeep Digital Deluxe\t5.387590\n57300\tAmnesia: The Dark Descent\t5.353191\n812140\tAssassin's Creed® Odyssey\t5.337534\n288470\tFable Anniversary\t5.336144\n972660\tSpiritfarer®\t5.321304\n466300\tPlanescape: Torment: Enhanced Edition\t5.316258\n434170\tThe Jackbox Party Pack 3\t5.305679\n973760\tThronebreaker: The Witcher Tales\t5.304965\n366090\tColony Survival\t5.298052\n995980\tFae Tactics\t5.249742\n1090200\tDestiny 2: Shadowkeep\t5.246860\n55230\tSaints Row: The Third\t5.240653\n320\tHalf-Life 2: Deathmatch\t5.222818\n471810\tDeath Squared\t5.222415\n374040\tPortal Knights\t5.198342\n740130\tTales of Arise\t5.192035\n247240\tV

In [131]:
for i , (col, imp) in enumerate(sorted(zip(X_pred.columns, model.feature_importances_), key=lambda x: x[1], reverse=True)[:50]):
    print(f'{i+1}:\t {col} - {imp}')

1:	 tags_emb_162 - 0.1768345683813095
2:	 short_desc_emb_77 - 0.1592406928539276
3:	 short_desc_emb_272 - 0.09468262642621994
4:	 name_emb_14 - 0.07069715112447739
5:	 name_emb_45 - 0.052143845707178116
6:	 name_emb_229 - 0.04693683609366417
7:	 short_desc_emb_163 - 0.04353907331824303
8:	 name_emb_308 - 0.038166243582963943
9:	 tags_emb_60 - 0.031079279258847237
10:	 tags_emb_76 - 0.030980655923485756
11:	 name_emb_44 - 0.027098599821329117
12:	 name_emb_10 - 0.024300318211317062
13:	 name_emb_340 - 0.023336512967944145
14:	 short_desc_emb_245 - 0.018827272579073906
15:	 short_desc_emb_43 - 0.018152710050344467
16:	 tags_emb_128 - 0.017257053405046463
17:	 tags_emb_324 - 0.015022580511868
18:	 name_emb_122 - 0.011898031458258629
19:	 short_desc_emb_273 - 0.010868777520954609
20:	 tags_emb_229 - 0.010383868589997292
21:	 short_desc_emb_117 - 0.006468922831118107
22:	 tags_emb_287 - 0.0063084508292376995
23:	 name_emb_120 - 0.006060250569134951
24:	 name_emb_56 - 0.004887982737272978
25

In [124]:
## Analysis

In [125]:
df = pd.read_excel('~/gdrive/video_games/reviews/reviews_and_wishlist.xlsx', skiprows=2)

df

,Rank,Game,AppID,Base Category,Tier 1 Category,Platform,Year(s) Played,Score,GoG Rating,Unnamed: 9,0.2626849304,Unnamed: 11,Unnamed: 12,0.7373150696,Unnamed: 14,Unnamed: 15
0,1.0,Dark Souls II: Scholar of the First Sin,335300.0,RPG,Adventure RPG,PC,2020.0,6.0,5.0,NaN,NaN,79,4.225,1.22,NaN,NaN
1,2.0,Dark Souls III,374320.0,RPG,Adventure RPG,PC,2020.0,6.0,5.0,NaN,Year,# of Games,Avg Rating,SD Rating,NaN,NaN
2,3.0,Dark Souls Remastered,570940.0,RPG,Adventure RPG,PC,2020.0,6.0,5.0,NaN,2020.0,58,4.31,1.2,NaN,NaN
3,4.0,Divinity: Original Sin 2,435150.0,RPG,Traditional RPG,PC,2020.0,6.0,5.0,NaN,2021.0,21,4.14,1.24,NaN,NaN
4,5.0,Factorio,427520.0,Management,Automation,PC,2020.0,6.0,5.0,NaN,2022.0,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,75.0,Outward,794260.0,RPG,Adventure RPG,PC,2021.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,76.0,Superbrothers: Sword & Sworcery EP,204060.0,Adventure,Point and Click,PC,2020.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,77.0,War Thunder,236390.0,Arena,Vehicle Shooter,PC,2020.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,78.0,"Warhammer 40,000: Mechanicus",673880.0,Strategy,Tactical Strategy,PC,2020.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
tmp = df.groupby(['Base Category']).agg({'Score': [np.mean, np.std, np.count_nonzero]})

tmp[('Score', 'ci_left')] = stats.truncnorm.interval(
    0.95,
    (1-tmp[('Score', 'mean')])/tmp[('Score', 'std')],
    (6-tmp[('Score', 'mean')])/tmp[('Score', 'std')],
    loc=tmp[('Score', 'mean')],
    scale=tmp[('Score', 'std')]
)[0]
tmp[('Score', 'ci_right')] = stats.truncnorm.interval(
    0.95,
    (1-tmp[('Score', 'mean')])/tmp[('Score', 'std')],
    (6-tmp[('Score', 'mean')])/tmp[('Score', 'std')],
    loc=tmp[('Score', 'mean')],
    scale=tmp[('Score', 'std')]
)[1]
tmp[('Score', 'prob_like')] = 1-stats.truncnorm.cdf(
    3.5,
    (1-tmp[('Score', 'mean')])/tmp[('Score', 'std')],
    (6-tmp[('Score', 'mean')])/tmp[('Score', 'std')],
    loc=tmp[('Score', 'mean')],
    scale=tmp[('Score', 'std')]
)
tmp[('Score', 'prob_dislike')] = stats.truncnorm.cdf(
    3.5,
    (1-tmp[('Score', 'mean')])/tmp[('Score', 'std')],
    (6-tmp[('Score', 'mean')])/tmp[('Score', 'std')],
    loc=tmp[('Score', 'mean')],
    scale=tmp[('Score', 'std')]
)

tmp = tmp.sort_values(('Score', 'prob_like'), ascending=False)

tmp

Score                                                        \
                   mean       std count_nonzero   ci_left  ci_right prob_like   
Base Category                                                                   
Action         4.272727  0.646670            11  3.004232  5.501697  0.883503   
Management     4.666667  1.032796             6  2.600668  5.875609  0.856759   
RPG            4.526316  1.428613            19  1.792589  5.877116  0.727495   
Arena          4.055556  1.109967            18  1.914737  5.746453  0.680923   
Adventure      4.250000  1.388730             8  1.673920  5.840981  0.678597   
Strategy       4.000000  1.348400            12  1.573968  5.791777  0.627055   
Other          2.000000       NaN             1       NaN       NaN       NaN   
Party          5.000000       NaN             1       NaN       NaN       NaN   
Platformer     6.000000       NaN             1       NaN       NaN       NaN   
Puzzle         4.000000       NaN             1       NaN       NaN       NaN   
Sandbox        4.000000       NaN             1       NaN       NaN       NaN   

                            
              prob_dislike  
Base Category               
Action            0.116497  
Management        0.143241  
RPG               0.272505  
Arena             0.319077  
Adventure         0.321403  
Strategy          0.372945  
Other                  NaN  
Party                  NaN  
Platformer             NaN  
Puzzle                 NaN  
Sandbox                NaN

In [127]:
tmp = df.groupby(['Base Category', 'Tier 1 Category']).agg({'Score': [np.mean, np.std, np.count_nonzero]})

tmp[('Score', 'ci_left')] = stats.truncnorm.interval(
    0.95,
    (1-tmp[('Score', 'mean')])/tmp[('Score', 'std')],
    (6-tmp[('Score', 'mean')])/tmp[('Score', 'std')],
    loc=tmp[('Score', 'mean')],
    scale=tmp[('Score', 'std')]
)[0]
tmp[('Score', 'ci_right')] = stats.truncnorm.interval(
    0.95,
    (1-tmp[('Score', 'mean')])/tmp[('Score', 'std')],
    (6-tmp[('Score', 'mean')])/tmp[('Score', 'std')],
    loc=tmp[('Score', 'mean')],
    scale=tmp[('Score', 'std')]
)[1]
tmp[('Score', 'prob_like')] = 1-stats.truncnorm.cdf(
    3.5,
    (1-tmp[('Score', 'mean')])/tmp[('Score', 'std')],
    (6-tmp[('Score', 'mean')])/tmp[('Score', 'std')],
    loc=tmp[('Score', 'mean')],
    scale=tmp[('Score', 'std')]
)
tmp[('Score', 'prob_dislike')] = stats.truncnorm.cdf(
    3.5,
    (1-tmp[('Score', 'mean')])/tmp[('Score', 'std')],
    (6-tmp[('Score', 'mean')])/tmp[('Score', 'std')],
    loc=tmp[('Score', 'mean')],
    scale=tmp[('Score', 'std')]
)

tmp = tmp.sort_values(('Score', 'prob_like'), ascending=False)

tmp

/Users/mcminnra/miniconda3/envs/sgr/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2128: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Users/mcminnra/miniconda3/envs/sgr/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2129: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
/Users/mcminnra/miniconda3/envs/sgr/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:1953: RuntimeWarning: divide by zero encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


Score                          \
                                         mean       std count_nonzero   
Base Category Tier 1 Category                                           
Management    Automation             5.500000  0.707107             2   
RPG           Traditional RPG        5.200000  0.836660             5   
Action        Narrative Action       4.500000  0.577350             4   
Adventure     Sandbox Adventure      4.333333  0.577350             3   
Management    Tycoon                 4.500000  0.707107             2   
Arena         Sports                 4.500000  0.707107             2   
Strategy      Tabletop-Like          5.000000  1.414214             2   
              Strategic Roguelike    4.250000  0.957427             4   
Action        Action Roguelike       4.250000  0.957427             4   
Arena         Corridor Shooter       4.000000  1.000000             3   
Adventure     Traditional Adventure  4.333333  1.527525             3   
RPG           MMORPG                 4.142857  1.345185             7   
Arena         MOBA                   4.000000  1.154701             4   
RPG           Adventure RPG          4.500000  1.974842             6   
Management    Colony Sim             4.000000  1.414214             2   
Arena         Battle Royale          3.750000  1.500000             4   
Strategy      Tactical Strategy      2.666667  1.527525             3   
Action        Immersive Sim          4.000000       NaN             1   
              Other Action           4.000000       NaN             1   
              Traditional Action     4.000000       NaN             1   
Adventure     Metrovania             6.000000       NaN             1   
              Point and Click        2.000000       NaN             1   
Arena         Autochess              5.000000       NaN             1   
              Card Battler           5.000000       NaN             1   
              Racing                 4.000000       NaN             1   
              Role-Based Shooter     5.000000       NaN             1   
              Vehicle Shooter        2.000000       NaN             1   
Other         Other                  2.000000       NaN             1   
Party         Social Deduction       5.000000       NaN             1   
Platformer    Puzzle Platformer      6.000000       NaN             1   
Puzzle        Puzzle                 4.000000       NaN             1   
RPG           Hack and Slash         4.000000       NaN             1   
Sandbox       Sandbox                4.000000       NaN             1   
Strategy      4X                     5.000000  0.000000             2   
              RTS                    3.000000       NaN             1   

                                                                                
                                      ci_left  ci_right prob_like prob_dislike  
Base Category Tier 1 Category                                                   
Management    Automation             4.032951  5.957628  0.996924     0.003076  
RPG           Traditional RPG        3.494687  5.933722  0.974615     0.025385  
Action        Narrative Action       3.367254  5.589546  0.958172     0.041828  
Adventure     Sandbox Adventure      3.201267  5.446743  0.925397     0.074603  
Management    Tycoon                 3.108938  5.725608  0.919995     0.080005  
Arena         Sports                 3.108938  5.725608  0.919995     0.080005  
Strategy      Tabletop-Like          2.132156  5.915513  0.812533     0.187467  
              Strategic Roguelike    2.365058  5.755405  0.775987     0.224013  
Action        Action Roguelike       2.365058  5.755405  0.775987     0.224013  
Arena         Corridor Shooter       2.052667  5.673163  0.685226     0.314774  
Adventure     Traditional Adventure  1.580176  5.859865  0.671939     0.328061  
RPG           MMORPG                 1.658788  5.819067  0.661769     0.338231  
Arena         MOBA                   1.802766  5.7